In [ ]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from scipy.stats import gmean

IMG_SIZE_PX = 50
SLICE_COUNT = 20

In [ ]:
stage1_data = np.load('stage1data-50-50-20.npy')

train_data = stage1_data[:]
print(train_data.shape)

train_features = []
train_labels = []


for i in range(0,train_data.shape[0]):
    train_features = train_features + [(train_data[i,0])[0]]
    train_labels = train_labels + [(train_data[i,1])[1]]
'''
for i in range(0,len(train_features)):
    for j in range(0,len(train_features[i])):
        for k in range(0,len((train_features[i])[j])):
                       if ((train_features[i])[j])[k] == -2000:
                           ((train_features[i])[j])[k] = 0
'''    
train_features = np.array(train_features)
train_labels = np.array(train_labels)

nsamples, nx, ny = train_features.shape
d2_train_features = train_features.reshape((nsamples,nx*ny))
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(d2_train_features)
d2_train_features = scaler.transform(d2_train_features)
'''
print(train_features.shape)
print(d2_train_features.shape)
print(train_labels.shape)

del train_data
del stage1_data
del train_features

In [ ]:
stage2_data = np.load('stage2data-50-50-20.npy')

print('Test Data')
test_data = stage2_data[:]
print(test_data.shape)

test_features = []

for i in range(0,test_data.shape[0]):
    test_features = test_features + [(test_data[i,0])[0]]
'''
for i in range(0,len(test_features)):
    for j in range(0,len(test_features[i])):
        for k in range(0,len((test_features[i])[j])):
                       if ((test_features[i])[j])[k] == -2000:
                           ((test_features[i])[j])[k] = 0    
'''    
test_features = np.array(test_features)
nsamples, nx, ny = test_features.shape
d2_test_features = test_features.reshape((nsamples,nx*ny))
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(d2_test_features)
d2_test_features = scaler.transform(d2_test_features)
'''
print('Test Data')
print(test_features.shape)
print(d2_test_features.shape)

del stage2_data
del test_data
del test_features

In [ ]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
xgdmat = xgb.DMatrix(d2_train_features,train_labels)

our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1}
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 1000)

In [ ]:
testdmat = xgb.DMatrix(d2_test_features)
test_pred_xgProb = final_gb.predict(testdmat)

pred = []
for i in range(0,test_pred_xgProb.shape[0]):
    pred = pred + [test_pred_xgProb[i]]

xgbPatients_df = pd.read_csv('..input/stage2_sample_submission.csv')
xgbPatients_df['cancer'] = pd.Series(pred)
xgbPatients_df.to_csv("..input//stage2_pred_xgb_6.csv",index=False)